Key Performance Indicators(KPIs):
    accuracy = magnitude of the difference btw forecast and demand
    bias = direction of the error on average = average error
    scaled bias = bias/average demand
    MAE = mean absolute error
    RMSE = root mean squared error
    



In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
from scipy.stats import norm

In [3]:
def kpi(df):
    average_demand = df.loc[df['Error'].notnull(),'Demand'].mean()
    bias = df['Error'].mean()
    relative_bias = bias/average_demand
    
    print('Bias: {:0.2f},{:.2%}'.format(bias,relative_bias))

In [4]:
def moving_average(demand, future_periods, n):
    columns = len(demand)
    
    demand = np.append(demand,[np.nan]*future_periods)
    forecast = np.full(columns + future_periods,np.nan)
    
    for t in range(n,columns):
        forecast[t] = np.mean(demand[t-n:t])
    
    forecast[t+1:] = np.mean(demand[t-n+1:t+1]) 
    
    df = pd.DataFrame.from_dict({'Demand':demand, 'Forecast':forecast, 'Error':demand-forecast})
    return df

In [6]:
demand = [40,29,65,90,120,138,170,145]

df = moving_average(demand, future_periods=4, n=3)
kpi(df)

Bias: 41.33,31.17%


Mean Absolute Error (MAE):
    average of the absolute values of the errors
    scaled MAE = MAE/average demand


In [12]:
def kpi2(df):
    average_demand = df.loc[df['Error'].notnull(),'Demand'].mean()
    bias = df['Error'].mean()
    scaled_bias = bias/average_demand
    print('Bias: {:0.2f},{:.2%}'.format(bias,scaled_bias))
    
    MAE = df['Error'].abs().mean()
    scaled_MAE = MAE/average_demand
    print('MAE: {:0.2f}, {:.2%}'.format(MAE,scaled_MAE))
    


In [14]:
demand = [40,29,65,90,120,138,170,145]
df = moving_average(demand, future_periods=4, n=3)
kpi2(df)

Bias: 41.33,31.17%
MAE: 41.33, 31.17%


In [15]:
df

,Demand,Forecast,Error
0,40.0,NaN,NaN
1,29.0,NaN,NaN
2,65.0,NaN,NaN
3,90.0,44.666667,45.333333
4,120.0,61.333333,58.666667
5,138.0,91.666667,46.333333
6,170.0,116.000000,54.000000
7,145.0,142.666667,2.333333
8,NaN,151.000000,NaN
9,NaN,151.000000,NaN


Root Mean Squared Error (RMSE):
    square root of the average of the squared errors
    scaled value = RMSE/average demand

In [16]:
def kpi3(df):
    average_demand = df.loc[df['Error'].notnull(),'Demand'].mean()
    print(f"average demand: {average_demand}")
    bias = df['Error'].mean()
    scaled_bias = bias/average_demand
    print('Bias: {:0.2f},{:.2%}'.format(bias,scaled_bias))
    
    MAE = df['Error'].abs().mean()
    scaled_MAE = MAE/average_demand
    print('MAE: {:0.2f}, {:.2%}'.format(MAE,scaled_MAE))
    
    RMSE = np.sqrt((df['Error']**2).mean())
    scaled_RMSE = RMSE/average_demand
    print('RMSE: {:0.2f}, {:.2%}'.format(RMSE,scaled_RMSE))
    
    

In [17]:
kpi3(df)

average demand: 132.6
Bias: 41.33,31.17%
MAE: 41.33, 31.17%
RMSE: 45.97, 34.67%


KPI Optimization:
    optimizing MAE aims at demand median
    optimizing RMSE aims at demand average

Key Factors:
    Bias: in real-world, median demand < mean demand (due to seasonal peaks)
          optimizing MAE = arriving at median = undershoots demand = bias
          so, RMSE guarantees an unbiased forecast

    Outliers: causes the mean to become a larger number, median stays the same
              optimizing RMSE = arriving at mean = overshoots demand
              so, MAE provides protection against outliers

    Intermittent Demand: MAE = bad KPI when half of the timeframe has no demand

In [20]:
# Outlier sensitivity example

demand = [16,8,10,8,7,13,6,6,7,13]
print(f"mean: {np.mean(demand)}, median: {np.median(demand)}")

demand_new = [16,8,10,8,7,13,6,6,7,13,100]
print(f"mean: {np.mean(demand_new)}, median: {np.median(demand_new)}")


mean: 9.4, median: 8.0
mean: 17.636363636363637, median: 8.0


In [25]:
# intermittent demand example

demand = [100,0,0] #weekly demand
forecast_mean = np.full(len(demand),np.mean(demand))
forecast_median = np.full(len(demand),np.median(demand))

error_mean = demand - forecast_mean
error_median = demand - forecast_median

error_mean_abs = np.abs(error_mean)
error_median_abs = np.abs(error_median)

error_mean_squared = error_mean ** 2
error_median_squared = error_median ** 2

print('Using mean as forecast:')
print(f"abs error: {np.sum(error_mean_abs)}, squared error: {np.sum(error_mean_squared)}")

print('Using median as forecast:')
print(f"abs error: {np.sum(error_median_abs)}, squared error: {np.sum(error_median_squared)}")



Using mean as forecast:
abs error: 133.33333333333334, squared error: 6666.666666666666
Using median as forecast:
abs error: 100.0, squared error: 10000.0
